# Devoir 2 - IFT6512

### Mathieu Beaudoin

In [1]:
using JuMP
using LinearAlgebra
using GLPK
using DataFrames

## Problem statement and data

We have 3 production facilities, indexed by $i=\{1,2,3\}$ and 5 demand locations, indexed by $j=\{1,...,5\}$. Unit production costs are 14, and the unit sale price is 24. Demand is not known in advance, and any surplus at a given location has to be disposed at a cost of 4 per unit. Production facilities have respective capacities of (500, 450, 650).  

Unit transport costs from facility $i$ ($f_i$) to demand point $j$ ($d_j$) are the following:  

|   | $d_1$| $d_2$| $d_3$| $d_4$| $d_5$|
|---|---|---|---|---|---|
|$f_1$| 2.49|5.21|3.76|4.85|2.07|
|$f_2$| 1.46|2.54|1.83|1.86|4.76|
|$f_3$| 3.26|3.08|2.60|3.76|4.45|  

Demand can take the following values:  

|j|Low|Medium|High|
|---|---|---|---|
|1|150|160|170|
|2|100|120|135|
|3|250|270|300|
|4|300|325|350|
|5|600|700|800|

with the following probabilities (we assume that demand levels at each location are independent of one another):  

|j|Low|Medium|High|
|---|---|---|---|
|1,2,3|0.25|0.50|0.25|
|4,5  |0.30|0.40|0.30|  

We have 15 decisions to make (how much to ship to each demand point from each production facility); our goal is to maximize expected profit

In [2]:
prod_cost = 14.
prod_capacity = [500; 450; 650]
sell_price = 24.
waste_cost = 4.

t_cost = [
    2.49 5.21 3.76 4.85 2.07;
    1.46 2.54 1.83 1.86 4.76;
    3.26 3.08 2.60 3.76 4.45
]
demand_level = [
    150 160 170;
    100 120 135;
    250 270 300;
    300 325 350;
    600 700 800
]
demand_pr = [
    .25 .5 .25;
    .25 .5 .25;
    .25 .5 .25;
    .3 .4 .3;
    .3 .4 .3;
]

5×3 Matrix{Float64}:
 0.25  0.5  0.25
 0.25  0.5  0.25
 0.25  0.5  0.25
 0.3   0.4  0.3
 0.3   0.4  0.3

## Scenario ventilation

Let us start by ventilating the possible states of nature, along with the probabilities. We first generate a matrix of indices, which we will use to build matrices of realizations and joint probabilities. We will refer to the random vector of demands as $\xi_k\in\mathbb{R}^5$, for $k=1,...,K$ (in the current context, $K=3^5=243$).

In [3]:
n, d = size(t_cost)

# Generate matrix of in
idx = zeros(n^d, d)
P = ones(n^d)
for j in 1:d
    for i in 1:n^d
        idx[i, j] = ceil(i / 3^(j-1)) % 3
        if idx[i, j] == 0
            idx[i, j] = 3
        end
    end
end
idx = convert(Matrix{Int64}, idx)

# Use the indices matrix to assemble disjointed scenarios and compute their probabilities
ξ = zeros(n^d, d)
marginal_P = ones(n^d, d)
for i = 1:n^d
    for j = 1:d
        ξ[i, j] = demand_level[j, idx[i, j]]
        marginal_P[i, j] = demand_pr[j, idx[i, j]]
    end
end
P = [prod(marginal_P[i, :]) for i=1:n^d]

# Stupid-check
sum(P)

1.0000000000000002

We probably don't need to worry that the probabilities vector doesn't sum exactly to one, since the result seems to be within a rounding error of it.

## First-stage program

The first-stage problem is the following: $\min\{c^\top x | Ax \le b, \ x\ge 0\}$. We represent $c$ and $x$ as 15-vectors; $b$ is the production capacity constraint. The $A$ matrix is the following:  
$$A = \begin{pmatrix}
    1 & 1 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \\
    0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & \\
    0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 1
  \end{pmatrix}$$

The $c$ vector is the sum of production and transport costs for each production facility/demand point combination, minus the selling price (we will assume, for now, that everything that can be produced can also be sold, and account for waste by adding optimality cuts later on).

In [4]:
# First-stage constraints
A = zeros(3, 15)
for i = 1:3
    j0 = (i-1) * 5 + 1
    A[i, j0:(j0+4)] = ones(5)
end
b = prod_capacity

# Cost vector
n = n * d
c = ones(n) * (prod_cost - sell_price) + reshape(transpose(t_cost), n, 1)

function first_stage(x)
    return dot(c, x)
end

# Test with completely random numbers
first_stage(rand(1:10, n, 1))

-577.08

## Master program

Using the multicut L-shaped algorithm, the master program has the following form:  
$$\min z = c^\top x + \sum_{k=1}^K \theta_k \\
\text{s.t.}
\quad Ax\le b
\\
D_{\ell}x \ge d_\ell, \quad \ell=1,...,r,
\\
E_{\ell(k)}x + \theta_k \ge e_{\ell(k)}, \quad \ell (k)=1,...,s_k,
\\
x\ge 0
$$  

For the current problem, there is no need to add any feasibility cuts ($D$, $d$), as the structure of the problem guarantees that $K_1\subseteq K_2$, i.e. any first-stage feasible solution is guaranteed to be second-stage feasible, as any shortage or surplus of shipped product relative to demand can be absorbed through foregone sales or paid disposal, respectively.  

Therefore, the only constraints we need to bake into the master program are the first-stage constraints; the rest will be added as we iterate through the L-shaped method.

In [5]:
function master(A, b, c, Ξ)
    
    m, n = size(A)
    K = size(Ξ)[1]
    
    model = Model(with_optimizer(GLPK.Optimizer))        
    @variable(model, x[1:n] >= 0)
    @variable(model, θ[1:K] >= -1e9)    
    for i = 1:m
        @constraint(model, (transpose(A)'x)[i] <= b[i])
    end    
    @objective(model, Min, first_stage(x) + sum(θ))    
    return model, x, θ
    
end

master (generic function with 1 method)

## Second-stage program

### Primal

In the first stage, we start off by assuming that everything we produce and ship will be sold. This simplifies the second-stage program, as we can focus this stage on minimizing waste, given a realization of $\xi$:  
$$\min\big\{q^\top y|Wy=h-Tx, y\ge 0 \big\}$$
where:
- $q$ and $y$ are 10-vectors, the first 5 positions of each representing the cost of sales lost due to insufficient delivery, and the rest representing the cost of wasted inventory ($q$ gives the coefficients, $y$ is the optimand)
    - since the negative cost of a successful sale is front-loaded in the first stage, we assign a null cost to sales lost due to insufficient inventory, but penalize excess inventory by retracting the sale price on the excess units, on top of the disposal cost;
- $W$ is the 5x10 recourse matrix:  
$$W = \begin{pmatrix}
    1 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & 0\\
    0 & 1 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0\\
    0 & 0 & 1 & 0 & 0 & 0 & 0 & -1 & 0 & 0\\
    0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & -1 & 0\\
    0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & -1
  \end{pmatrix}$$  
  
  
- $h$ is the 5-vector of demand associated with a given realization of $\xi$;
- $T$ is the indicator matrix summing the quantities shipped to each demand location:  

$$T = \begin{pmatrix}
    1 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & \\
    0 & 1 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & \\
    0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & \\
    0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 0 & \\
    0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1
  \end{pmatrix}$$

In [6]:
q = [zeros(5); ones(5) * (sell_price + waste_cost)]
I = diagm(ones(5))
W = [I -I]
T = [I I I]

5×15 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0

### Dual

To implement the L-shaped algorithm, we need to define the dual second-stage problem - defining the primal problem is mainly a stepping-stone to the dual. We introduce $\lambda$ as the dual variable vector. Considering the primal had the form $\min\{q^\top y|Wy=z, y\ge 0\}$, the dual becomes $\max\{\lambda^\top z|W^\top\lambda\le q\}$, with $\lambda$ unconstrained.

In [7]:
function second_stage_dual(x, q, W, T, ξ_k)
    
    m, n = size(W)
    
    model_dual = Model(with_optimizer(GLPK.Optimizer))
    @variable(model_dual, λ[1:m])
    for i = 1:n
        @constraint(model_dual, (W'λ)[i] <= q[i])
    end
    
    z = ξ_k - T * x
    @objective(model_dual, Max, λ'z)
    
    optimize!(model_dual)
    
    return value.(λ), objective_value(model_dual)
    
end

second_stage_dual (generic function with 1 method)

## L-Shaped method (multicut)

We now implement the overall algorithm.

In [8]:
function lshaped(A, b, c, Ξ, q, W, T, P, max_iter=10)
    
    K = size(Ξ)[1]
    base_θ = ones(K) * -Inf
    stop = false
    
    model, x, θ = master(A, b, c, Ξ)
    
    for iter = 1:max_iter
        
        println("\nIteration #", iter)
        
        optimize!(model)
        status = termination_status(model)
        if status != MOI.OPTIMAL
            println("\nERROR - status: ", status)
            return model, status
        end
        
        x_star = value.(x)
        base_θ = value.(θ)
        E = zeros(K, size(T)[2])
        e = zeros(K)
        
        println("\toptimal value: ", objective_value(model),
                "\n\tx vector: ", [round(_x * 10) / 10 for _x in x_star])
        
        for k = 1:K
            
            ξk = Ξ[k, :]
            
            # Get the simplex multipliers and dual program objective value
            λ, θ_v = second_stage_dual(x_star, q, W, T, ξk)
            
            # Check for optimality before going any further
            p = P[k]
            if base_θ[k] >= p * θ_v
                stop = true
                break
            end
            
            # Build optimality cuts
            E[k, :] = p * λ'T
            e[k] = p * λ'ξk            
            @constraint(model, dot(E[k, :], x) + θ[k] >= e[k])
            
        end
        
        if stop
            break
        end
        
    end
    
    return model, value.(x), objective_value(model)
        
end

m, x, optimum = lshaped(A, b, c, ξ, q, W, T, P)


Iteration #1
	optimal value: -2.43000012618e11
	x vector: [0.0, 0.0, 0.0, 0.0, 500.0, 450.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 650.0, 0.0, 0.0]

Iteration #2
	optimal value: -24236.000000596046
	x vector: [0.0, 0.0, 0.0, 0.0, 500.0, 0.0, 0.0, 0.0, 450.0, 0.0, 0.0, 650.0, 0.0, 0.0, 0.0]

Iteration #3
	optimal value: -21833.562500596046
	x vector: [0.0, 0.0, 0.0, 0.0, 500.0, 0.0, 0.0, 0.0, 0.0, 450.0, 0.0, 0.0, 0.0, 0.0, 650.0]

Iteration #4
	optimal value: -13278.82500163572
	x vector: [0.0, 0.0, 0.0, 0.0, 205.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Iteration #5
	optimal value: -10985.178124248036
	x vector: [0.0, 0.0, 0.0, 0.0, 500.0, 15.0, 0.0, 0.0, 435.0, 0.0, 0.0, 0.0, 415.0, 0.0, 115.0]

Iteration #6
	optimal value: -10973.178124060707
	x vector: [0.0, 0.0, 0.0, 0.0, 500.0, 165.0, 0.0, 0.0, 285.0, 0.0, 0.0, 150.0, 265.0, 0.0, 115.0]

Iteration #7
	optimal value: -10887.349999967077
	x vector: [0.0, 0.0, 0.0, 0.0, 500.0, 130.0, 0.0, 0.0, 320.0, 0.0, 40.0, 100.0, 250.0, 0.

(A JuMP Model
Minimization problem with:
Variables: 258
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 1460 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 258 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: x, θ, [0.0, 0.0, 0.0, 0.0, 500.0, 129.99999977293476, 0.0, 0.0, 320.00000022706524, 0.0, 39.999999318803994, 100.00000000000009, 249.9999999999999, 0.0, 100.0], -10887.349999967077)

In [9]:
function result_as_df(result, orig_shape)
    if length(size(result)) == 1 || size(result)[2] == 1
        result = transpose(reshape(result, orig_shape[2], orig_shape[1]))
    end
    return DataFrame(Dict(string("d", j) => result[:, j] for j=1:5))
end

result_as_df(x, size(t_cost))

,d1,d2,d3,d4,d5
,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,500.0
2,130.0,0.0,0.0,320.0,0.0
3,40.0,100.0,250.0,0.0,100.0


In [10]:
result_as_df(t_cost, size(t_cost))

,d1,d2,d3,d4,d5
,Float64,Float64,Float64,Float64,Float64
1,2.49,5.21,3.76,4.85,2.07
2,1.46,2.54,1.83,1.86,4.76
3,3.26,3.08,2.6,3.76,4.45


In [11]:
DataFrame(Dict(
        "Mean Demand" => [sum(P .* ξ[:, j]) for j=1:5],
        "Optimal Shipment" => T * x
))

,Mean Demand,Optimal Shipment
,Float64,Float64
1,160.0,170.0
2,118.75,100.0
3,272.5,250.0
4,325.0,320.0
5,700.0,600.0


By comparing the optimal shipments to the table of shipping costs, we can see the results make a lot of sense: each facility heavily favors destinations that have a relatively low shipping costs, while considering that for some locations, it can be more profitable to serve the limited demand from a higher-cost facility to avoid the latter having to use its capacity on even higher-cost locations.

## L-Shaped method with regularization

We will use a slightly modified version of the regularization term, using the L1 distance (rather than L2) between $x^\nu$ and $a^\nu$. The reason we are forced to do this is because our current solver will not let us work with quadratic problems.

In [12]:
function lshaped_reg(A, b, c, Ξ, q, W, T, P, x0, α=0.5, max_iter=10, verbose=0)
    
    K = size(Ξ)[1]
    base_θ = ones(K) * -Inf
    stop = false
    a_ν = x0     # Baseline solution
    Qx = -1e10
    e = ones(K)
    E = ones(K, size(T)[2])
    
    # Expected recourse value, initial solution
    Q = 0
    margins = zeros(K)
    for k = 1:K
        λ, θ_v = second_stage_dual(a_ν, q, W, T, Ξ[k, :])
        Q += P[k] * θ_v
    end
    
    # Initializing ahead of for loops, because Julia has weird scoping
    x_star = zeros(size(x0))
    right_hand = 0
    left_hand = 0
    
    model, x, θ = master(A, b, c, Ξ)
    
    # Add regularization term
    n = size(x)[1]    
    @variable(model, Δ_neg[1:n] >= 0)    
    @variable(model, Δ_pos[1:n] >= 0)
    @variable(model, a[1:n] >= 0)
    for i = 1:n
        @constraint(model, Δ_pos[i] - Δ_neg[i] == x[i] - a[i])
        JuMP.fix(a[i], a_ν[i]; force = true)
    end
    @objective(model, Min, first_stage(x) + sum(θ) + α * (sum(Δ_neg) + sum(Δ_pos)))
    
    for iter = 1:max_iter
        
        if verbose > 0
            println("\nIteration #", iter)
        end
        
        optimize!(model)
        status = termination_status(model)
        if status != MOI.OPTIMAL
            println("\nERROR - status: ", status)
            return model, status
        end        
        x_star = value.(x)
        base_θ = value.(θ)
        
        if e'base_θ == (c'a_ν)[1] + Q
            println("\nBREAK 1")
            break  # a_ν is optimal
        end
        
        if verbose > 0
            println("\toptimal value: ", objective_value(model),
                    "\n\tx vector: ", [round(_x * 10) / 10 for _x in x_star])
        end
        
        for k = 1:K
            
            ξk = Ξ[k, :]
            
            # Get the simplex multipliers and dual program objective value
            λ, θ_v = second_stage_dual(x_star, q, W, T, ξk)
            
            # Check for optimality before going any further
            p = P[k]
            margins[k] = p * θ_v
            if base_θ[k] >= margins[k]
                stop = true
                a_ν = x_star        
                break
            end
            
            # Build optimality cuts
            E[k, :] = p * λ'T
            e[k] = p * λ'ξk            
            @constraint(model, dot(E[k, :], x) + θ[k] >= e[k])
            
        end
        
        # Check if a_ν needs updating
        Qx = sum(margins)
        right_hand = (c'x_star)[1] + Qx
        left_hand = (c'a_ν)[1] + Q
        if verbose > 0
            println("\tQ: ", Q, "\tQx: ", Qx,
                    "\n\tright_hand: ", right_hand,
                    "\n\tleft_hand: ", left_hand)
        end
        if right_hand <= left_hand
            a_ν = x_star            
            for i = 1:n
                JuMP.fix(a[i], a_ν[i]; force = true)
            end
            Q = Qx
        end
        
        if stop
            break
        end
        
    end
    
    return model, a_ν, objective_value(model)
        
end

α = 0.1
x0 = zeros(15)
# A, b, c, Ξ, q, W, T, P, x0, α=0.5, max_iter=10, verbose=0
maxiter = 20
verbosity = 1
m_reg, x_reg, optimum_reg = lshaped_reg(A, b, c, ξ, q, W, T, P, x0, α, maxiter, verbosity)


Iteration #1
	optimal value: -2.43000012458e11
	x vector: [0.0, 0.0, 0.0, 0.0, 500.0, 450.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 650.0, 0.0, 0.0]
	Q: 0.0	Qx: 18690.0
	right_hand: 6072.0
	left_hand: 0.0

Iteration #2
	optimal value: -24076.000000596046
	x vector: [0.0, 0.0, 0.0, 0.0, 500.0, 0.0, 0.0, 0.0, 450.0, 0.0, 0.0, 650.0, 0.0, 0.0, 0.0]
	Q: 0.0	Qx: 18375.0
	right_hand: 6249.0
	left_hand: 0.0

Iteration #3
	optimal value: -21673.562501192093
	x vector: [0.0, 0.0, 0.0, 0.0, 500.0, 0.0, 0.0, 0.0, 0.0, 450.0, 0.0, 0.0, 0.0, 0.0, 650.0]
	Q: 0.0	Qx: 25200.0
	right_hand: 15269.5
	left_hand: 0.0

Iteration #4
	optimal value: -13258.143750724339
	x vector: [0.0, 0.0, 0.0, 0.0, 200.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
	Q: 0.0	Qx: 0.0
	right_hand: -1586.000018165225
	left_hand: 0.0

Iteration #5
	optimal value: -10857.990624354943
	x vector: [0.0, 0.0, 0.0, 0.0, 500.0, 25.0, 0.0, 0.0, 425.0, 0.0, 0.0, 0.0, 395.0, 0.0, 105.0]
	Q: 0.0	Qx: 6272.0000094837615
	right_hand: -4871.74998

(A JuMP Model
Minimization problem with:
Variables: 303
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 15 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 1231 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 15 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 288 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: a, x, Δ_neg, Δ_pos, θ, [0.0, 0.0, 0.0, 0.0, 500.0, 157.50000029802322, 0.0, 0.0, 292.4999997019768, 0.0, 0.0, 132.50000159891837, 262.49999978712617, 0.0, 105.00000069538763], -10918.34062265891)

In [13]:
alphas = [0; .05; .1; .2; .5; 1.]
headers = ["no-reg"]
d = size(alphas)[1]
X = zeros(15, d)
optima = zeros(d)
for i = 1:d
    α = alphas[i]
    println("α = ", α)
    m_reg, X[:, i], optima[i] = lshaped_reg(A, b, c, ξ, q, W, T, P, x0, α)
    push!(headers, string("alpha=", α))
end

α = 0.0
α = 0.05
α = 0.1
α = 0.2
α = 0.5
α = 1.0


## Discussion of results

I wasn't quite able to implement the exact regularization technique discussion in the lecture slides and in Birge & Louveaux's textbook, since the GLPK solver does not accept quadratic constraints or objectives. However, using the L1 distance for regularization is quite close. In the circumstances of this problem, since most values are either zero or much greater than one, L1 regularization's effect is likely to be considerably smaller than L2's.  

To look at the impact of different $\alpha$ regularization hyperparameters, I ran several optimizations with different values of $\alpha$. The following tables present the optimal decisions and objective values for each setting.

In [14]:
X = [x X]
DataFrame(Dict(
    headers[j] => X[:, j] for j = 1:(d+1)
))

,alpha=0.0,alpha=0.05,alpha=0.1,alpha=0.2,alpha=0.5,alpha=1.0,no-reg
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,500.0,500.0,500.0,500.0,500.0,500.0,500.0
6,130.0,162.5,157.5,157.5,155.0,150.0,130.0
7,0.0,0.0,0.0,0.0,0.0,1.12785e-14,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,320.0,287.5,292.5,292.5,295.0,300.0,320.0


In [15]:
opt = [optimum optima']
DataFrame(Dict(
    headers[j] => opt[:, j] for j = 1:(d+1)
))

,alpha=0.0,alpha=0.05,alpha=0.1,alpha=0.2,alpha=0.5,alpha=1.0,no-reg
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-10887.3,-10878.4,-10918.3,-10865.3,-10708.7,-10457.3,-10887.3


Unsurprisingly, the case where $\alpha=0$ produces the exact same results as the unregularized L-shaped algorithm. Also, the objective value generally gets worse as $\alpha$ increases, which fits well with one of regularization's goals, which is to reduce the risk of overfit, at the cost of the model performing less well on in-sample data. However, in the case of the L-shaped method, regularization also serves a numerical stability purpose, which perhaps explains the fact that one version of the model with *just a little bit* of regularization ($\alpha=0.1$) performed better than the unregularized version.